In [6]:
%reload_ext autoreload
%autoreload 2
#%reload_ext notexbook
#%texify

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import random
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
import matplotlib
import pandas as pd

%matplotlib qt5

In [8]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record
import seaborn as sns
import pandas as pd


ImportError: DLL load failed while importing spline: The specified module could not be found.

In [87]:
model_path = Path('/home/axtr7550/Chromosome_project/real_images/for_axel/model_rotated_45_venus_long_run_700_range.pkl')

In [88]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

INITIATING CPU IMPLEMENTATION


In [89]:
chromo_model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [90]:
chromo_model.evaluation_params['ground_truth']

[[1,
  1,
  27463.76953125,
  6984.809684753418,
  58.33728313446045,
  2466.4952158927917],
 [2,
  1,
  27067.937622070312,
  7112.922248840332,
  -639.0844702720642,
  2739.3634915351868],
 [3,
  1,
  33653.31756591797,
  7093.717613220215,
  24.959683418273926,
  2591.2692546844482],
 [4,
  1,
  34095.47546386719,
  7200.852165222168,
  78.018057346344,
  1854.9737334251404],
 [5,
  1,
  36916.861877441406,
  7260.620803833008,
  9.543609619140625,
  1242.2350645065308],
 [6,
  1,
  39081.61102294922,
  7254.732856750488,
  -373.4238922595978,
  2268.0033445358276],
 [7,
  1,
  33532.005920410156,
  7480.944480895996,
  -78.50396633148193,
  1374.5031952857971],
 [8,
  1,
  35678.65234375,
  7605.734939575195,
  -202.42498517036438,
  874.411404132843],
 [9,
  1,
  46118.79333496094,
  7574.54216003418,
  108.15801620483398,
  2262.0611786842346],
 [10,
  1,
  42884.75372314453,
  7667.521438598633,
  238.85810375213623,
  784.4618260860443],
 [11,
  1,
  46287.35534667969,
  7799.6

In [68]:
plot_train_record(chromo_model)

In [401]:
iterations = [key for key, value in chromo_model.recorder['count_loss'].items()]
count_loss = [value for key, value in chromo_model.recorder['count_loss'].items()]
loc_loss = [value for key, value in chromo_model.recorder['loc_loss'].items()]
bg_loss = [value for key, value in chromo_model.recorder['bg_loss'].items()]
cross_entropy_loss = [value for key, value in chromo_model.recorder['P_locs_error'].items()]

In [402]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].plot(iterations, count_loss)
ax[0, 0].set_xlabel('iterations')
ax[0, 0].set_ylabel('count loss')
ax[0, 1].plot(iterations, loc_loss)
ax[0, 1].set_xlabel('iterations')
ax[0, 1].set_ylabel('localization loss')
ax[1, 0].plot(iterations, bg_loss)
ax[1, 0].set_xlabel('iterations')
ax[1, 0].set_ylabel('noise free PSF loss')
ax[1, 1].plot(iterations, cross_entropy_loss)
ax[1, 1].set_xlabel('iterations')
ax[1, 1].set_ylabel('cross entropy loss')
plt.show()

In [69]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assess, assemble_full_img_predictions, limited_matching

#### For simulated data

In [91]:
eval_img = chromo_model.evaluation_params['eval_imgs']

In [92]:
plt.figure()
plt.imshow(eval_img[0])
plt.colorbar()
plt.show()

In [93]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [94]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.1,
                                             nms_threshold=0.1, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [95]:
chromo_model.evaluation_params.keys()

dict_keys(['number_images', 'use_cell_masks', 'density_in_cells', 'cell_masks_dir', 'cell_masks_filetype', 'edt_noise_map_path', 'eval_size', 'molecules_per_img', 'margin_empty', 'candidate_threshold', 'nms_threshold', 'print_result', 'batch_size', 'padded_background', 'eval_filenames', 'eval_imgs', 'ground_truth', 'fov_size'])

In [51]:
len(preds_tmp)

20641

##### Getting values of predicted and true x, y, z, i to plot distributions

In [96]:
ground_truth = chromo_model.evaluation_params['ground_truth']

In [97]:
len(ground_truth)

21494

In [98]:
min_int = np.percentile(np.array(ground_truth)[:, -1], 25)

In [99]:
min_int

1314.3160343170166

In [100]:
min_int = 700


In [101]:
# Not sure which is x and which is y
GT_df = pd.DataFrame(ground_truth,columns=['emitter_index', 'image_index', 'y_position', 'x_position', 'z', 'phot'] )
GT_df


,emitter_index,image_index,y_position,x_position,z,phot
0,1,1,27463.769531,6984.809685,58.337283,2466.495216
1,2,1,27067.937622,7112.922249,-639.084470,2739.363492
2,3,1,33653.317566,7093.717613,24.959683,2591.269255
3,4,1,34095.475464,7200.852165,78.018057,1854.973733
4,5,1,36916.861877,7260.620804,9.543610,1242.235065
...,...,...,...,...,...,...
21489,715,30,66409.691467,66650.708008,330.102682,1498.369664
21490,716,30,72171.677246,66672.432861,-188.064301,913.723826
21491,717,30,62953.638611,66745.446777,-634.255838,2422.890186
21492,718,30,67431.485596,66741.217651,-413.328969,1757.626712


In [102]:
fig, ax = plt.subplots()

sns.scatterplot(data=GT_df, x='z', y='phot')

<Axes: xlabel='z', ylabel='phot'>

In [103]:
perf_dict, matches = limited_matching(ground_truth, preds_tmp, min_int=min_int, limited_x=[0, fov_size[0]],
                                     limited_y=[0, fov_size[1]], border=450, print_res=True, tolerance=250,
                                      tolerance_ax=np.inf)

FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21464 ,preds: 20730
Recall: 0.958
Precision: 0.992
Jaccard: 95.135
RMSE_lat: 37.185
RMSE_ax: 70.671
RMSE_vol: 79.856
Jaccard/RMSE: 2.558
Eff_lat: 62.498
Eff_ax: 64.331
Eff_3d: 63.415
FN: 893.0 FP: 159.0


### Preds list has the following in order

    1. counter of the molecule 
    2. image number used to index into the number of the image in the prediction arrays
    3. x position in nm where 0 is top left corner
    4. y position in nm where 0 is top left corner
    5. z position in nm where 0 is from the reference 0 nm in height
    6. photon counts 
    7. probability afer nms
    8. x_sigma in nm 
    9. y_sigma in nm
    10. z_sigma in nm
    11. photon_counts_sigma
    12. x offset
    13. y offset


In [104]:
df_predictions = pd.DataFrame(preds_tmp, columns=["counter of the molecule","image number", "x","y","z","phot", "prob_post_nms","x_sigma","y_sigma","z_sigma","phot_sigma", "x_offset","y_offset"])

In [105]:
fig, ax = plt.subplots()
sns.scatterplot(data=df_predictions, x="phot", y="z_sigma", alpha=0.1)

<Axes: xlabel='phot', ylabel='z_sigma'>

### Plot distributions of true vs predicted to check for any biases on the full scale

matches is a matrix where rows correspond to a single localization,
columns are as follows. There are 12 columns, I only know what the first 8 are (x and y are in large FOV nm converted)

    1. ground truth x 
    2. ground truth y
    3. ground truth z
    4. ground truth photon counts
    5. predicted x
    6. predicted y
    7. predicted z
    8. predicted photon counts
    10. **Probably** z_sigma

In [106]:
df_matches =pd.DataFrame(matches,columns= ['GT_x', 'GT_y','GT_z', 'GT_phot', 'pred_x','pred_y','pred_z', 'pred_phot', 'Prob_nms','z_sigma', '', ''])
df_matches 


,GT_x,GT_y,GT_z,GT_phot,pred_x,pred_y,pred_z,pred_phot,Prob_nms,z_sigma,,
0,49142.312927,63085.296936,137.580168,2487.176478,49142.992188,63085.472656,114.445511,2479.747314,0.982278,37.689507,124.745834,0.046018
1,27182.496948,16040.051270,-179.002303,1843.142509,27183.798828,16039.704102,-161.519714,2732.546143,0.890484,50.828194,186.499603,0.212280
2,67858.936768,67033.987427,-275.967181,2382.654548,67859.054688,67035.351562,-232.250412,2373.035645,1.053917,29.042723,111.087151,-0.014536
3,46359.341736,62809.459534,-138.311410,1218.225539,46357.917969,62808.792969,-155.363129,1184.884399,1.005178,48.053169,90.894981,0.198712
4,69287.286377,8606.813889,-657.960403,2516.951144,69285.703125,8607.421875,-686.851868,2680.502686,0.780233,9.943767,116.436295,-0.066049
...,...,...,...,...,...,...,...,...,...,...,...,...
20566,54095.412903,50655.598145,56.156242,778.229713,54157.371094,50769.476562,159.966339,1527.234985,0.732340,97.837708,173.363815,0.190312
20567,35461.792603,50062.187500,-381.836092,891.326129,35434.808594,49932.898438,-210.348328,969.874512,0.818743,72.813850,113.276436,0.150916
20568,37834.879761,7556.911545,-183.393133,1245.408118,37942.585938,7464.794922,-331.220215,1344.321289,0.699327,42.350517,116.527092,-0.267932
20569,68780.386353,17712.813416,-464.597958,1266.605258,68643.796875,17758.250000,-474.600647,1671.723267,0.705827,48.662182,225.065384,0.058438


In [107]:
save_path = '/home/axtr7550/Chromosome_project/real_images/'

df_matches.to_csv(save_path + 'rotated_700_matches.csv', index=False)
df_predictions.to_csv(save_path + 'rotated_700_predictions.csv', index=False)
GT_df.to_csv(save_path + 'rotated_700_GT.csv', index=False)

In [86]:
fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='GT_phot',hue='Prob_nms', palette='viridis', alpha=0.3)


<Axes: xlabel='GT_z', ylabel='GT_phot'>

In [169]:
fig, ax = plt.subplots()

sns.histplot(data=df_matches, x='Prob_nms')

<Axes: xlabel='Prob_nms', ylabel='Count'>

In [166]:
from scipy.stats import linregress

for i in range(-500, 600, 100):
    lower_bound = i
    upper_bound = i + 200
    data_df= df_matches[(df_matches['z_sigma'] > 0) & (df_matches['pred_z'] > lower_bound)& (df_matches['pred_z'] < upper_bound)]
    jp = sns.jointplot(data=data_df, x="pred_z", y="GT_z", kind="reg")


    # Customize the scatterplot color
    scatter_color = 'lightblue'
    jp.ax_joint.collections[0].set_facecolor(scatter_color)
    jp.ax_joint.collections[0].set_edgecolor(scatter_color)

    # Customize the regression line color
    reg_line_color = 'red'
    jp.ax_joint.lines[0].set_color(reg_line_color)

    slope, intercept, r_value, p_value, std_err = linregress(data_df['pred_z'], data_df['GT_z'])

    # Compute the uncertainty bounds for the regression line
    x_vals = np.linspace(data_df['pred_z'].min(), data_df['pred_z'].max(), 100)
    y_pred = slope * x_vals + intercept
    y_upper = y_pred + 1.96 * std_err
    y_lower = y_pred - 1.96 * std_err

    # Plot the regression line with uncertainty bounds
    jp.ax_joint.plot(x_vals, y_pred, color='red', label='Regression Line')
    jp.ax_joint.fill_between(x_vals, y_lower, y_upper, color='lightcoral', alpha=0.3, label='95% CI')

    jp.ax_joint.set_ylim(-500, 500)

    # Show the legend
    jp.ax_joint.legend([f'r={r_value:.2f}', f'p={p_value:.3f}', f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}"])
    jp.fig.suptitle(f"Not rotated")


type: need at least one array to concatenate

In [84]:

plt.close('all')

In [86]:
lower_bound = 125
upper_bound = 175
phot_low = 700
phot_high = 1500 
data_df= df_matches[(df_matches['GT_phot'] >= phot_low) & (df_matches['GT_phot']<=phot_high) & (df_matches['pred_z'] > lower_bound)& (df_matches['pred_z'] < upper_bound)]
jp = sns.jointplot(data=data_df, x="pred_z", y="GT_z", kind="reg")


# Customize the scatterplot color
scatter_color = 'lightblue'
jp.ax_joint.collections[0].set_facecolor(scatter_color)
jp.ax_joint.collections[0].set_edgecolor(scatter_color)

# Customize the regression line color
reg_line_color = 'red'
jp.ax_joint.lines[0].set_color(reg_line_color)

slope, intercept, r_value, p_value, std_err = linregress(data_df['pred_z'], data_df['GT_z'])

# Compute the uncertainty bounds for the regression line
x_vals = np.linspace(data_df['pred_z'].min(), data_df['pred_z'].max(), 100)
y_pred = slope * x_vals + intercept
y_upper = y_pred + 1.96 * std_err
y_lower = y_pred - 1.96 * std_err

# Plot the regression line with uncertainty bounds
jp.ax_joint.plot(x_vals, y_pred, color='red', label='Regression Line')
jp.ax_joint.fill_between(x_vals, y_lower, y_upper, color='lightcoral', alpha=0.3, label='95% CI')
jp.ax_joint.errorbar(data_df["pred_z"], data_df["GT_z"], yerr=data_df['z_sigma']*1.96, fmt='o', color='lightblue', ecolor='gray', capsize=3)
jp.ax_joint.set_ylim([-500, 500])
# Show the legend
jp.ax_joint.legend([f'r={r_value:.2f}', f'p={p_value:.3f}', f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}"])
jp.fig.suptitle(f'True position of predicted dots with {lower_bound}<z<{upper_bound}, {phot_low}<photon count<{phot_high}')  # y=1.02 adjusts the title position slightly above the plot


Text(0.5, 0.98, 'True position of predicted dots with 125<z<175, 700<photon count<1500')

In [ ]:
sampled_z = [item[4] for item in ground_truth]
sampled_ph = [item[5] for item in ground_truth]

In [51]:
gt_x, pred_x = matches[:, 0], matches[:, 4]
residual_x = (gt_x - pred_x)
gt_y, pred_y = matches[:, 1], matches[:, 5]
residual_y = (gt_y - pred_y)
gt_z, pred_z = matches[:, 2], matches[:, 6]
residual_z = (gt_z - pred_z)
gt_ph, pred_ph = matches[:, 3], matches[:, 7]
residual_ph = (gt_ph, pred_ph)

In [52]:
matplotlib.style.use('ggplot')


In [87]:
fig = plt.figure()

sns.scatterplot(data=df_matches, x="pred_z", y='pred_phot', hue='GT_z',size='GT_phot',  palette='coolwarm', alpha=0.5)
#sns.scatterplot(data=df, x="GT_z", y='GT_phot')


<Axes: xlabel='pred_z', ylabel='pred_phot'>

In [88]:

fig = plt.figure()

sns.scatterplot(data=df_matches, x="GT_z", y='GT_phot', hue='pred_z',size='pred_phot',  palette='coolwarm', alpha=0.5)


<Axes: xlabel='GT_z', ylabel='GT_phot'>

In [149]:

def plot_subset(df, ax, z_low, z_high, phot_low, phot_high):
    data_df = df[(df['GT_z'] <=z_high)&(df['GT_z']>=z_low) & (df['GT_phot'] >= phot_low) & (df['GT_phot']<=phot_high)].reset_index()

    sns.scatterplot(ax=ax, data=data_df, x="GT_z", y='GT_phot', alpha=0.3, label=f'True positions {z_low}<z<{z_high} and {phot_low}<phot<{phot_high}')
    sns.scatterplot(ax=ax, data=data_df, x="pred_z", y='pred_phot',alpha=0.8, c='red', label='Corresponding predicted positions')

    z_start = data_df.loc[:,'GT_z']
    phot_start = data_df.loc[:, 'GT_phot']
    dz = data_df.loc[:, 'pred_z']-data_df.loc[:, 'GT_z']
    dphot = data_df.loc[:, 'pred_phot']-data_df.loc[:, 'GT_phot' ]

    #for i in range(len(data_df)):
     #   ax.arrow(z_start[i], phot_start[i], dz[i], dphot[i] ,width=0.01, facecolor='white', edgecolor='white', alpha=0.5)

    return ax

fig, ax = plt.subplots(figsize=(10,10))

sns.scatterplot(ax = ax, data=df_matches, x="pred_z", y='pred_phot',  palette='coolwarm', alpha=0.5, c='green', label='All predicted positions')

ax.set_xlabel("Z")
ax.set_ylabel("Photon count")

z_low = 400
z_high = 500
phot_low = 700  
phot_high = 2000

ax = plot_subset(df_matches, ax, z_low, z_high, phot_low, phot_high)
#ax = plot_subset(df, ax, -350, -300, 0, 2000)


/tmp/ipykernel_5067/2802139919.py:19: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.scatterplot(ax = ax, data=df_matches, x="pred_z", y='pred_phot',  palette='coolwarm', alpha=0.5, c='green', label='All predicted positions')


In [91]:

def plot_subset_flipped(df, ax, z_low, z_high, phot_low, phot_high):
    data_df = df[(df['pred_z'] <=z_high)&(df['pred_z']>=z_low) & (df['pred_phot'] >= phot_low) & (df['pred_phot']<=phot_high)].reset_index()

    sns.scatterplot(ax=ax, data=data_df, x="pred_z", y='pred_phot', alpha=0.3, label=f'Predicted positions {z_low}<z<{z_high} and {phot_low}<phot<{phot_high}')
    sns.scatterplot(ax=ax, data=data_df, x="GT_z", y='GT_phot',  palette='coolwarm', alpha=0.8, c='red', label=f'Corresponding GT positons')

    return ax

fig, ax = plt.subplots(figsize=(10,10))

sns.scatterplot(ax = ax, data=df_matches, x="GT_z", y='GT_phot',   alpha=0.5, c='green', label=f'All true positions')

ax.set_xlabel("Z")
ax.set_ylabel("Photon count")

z_low = 150
z_high = 250
phot_low = 700  
phot_high = 1200

ax.set_title(f"Spread of true values of predictions between {z_low}nm and {z_high}nm")


ax = plot_subset_flipped(df_matches, ax, z_low, z_high, phot_low, phot_high)
#ax = plot_subset(df, ax, -350, -300, 0, 2000)


/tmp/ipykernel_5067/2059549628.py:5: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.scatterplot(ax=ax, data=data_df, x="GT_z", y='GT_phot',  palette='coolwarm', alpha=0.8, c='red', label=f'Corresponding GT positons')


In [214]:
df_matches['z_error'] =  df_matches['pred_z'] - df_matches['GT_z']
df_matches['phot_error'] = df_matches['pred_phot'] - df_matches['GT_phot']
df_matches['abs_z_error'] = abs(df_matches['z_error'])

fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='z_error')
fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='phot_error')


<Axes: xlabel='GT_z', ylabel='phot_error'>

In [38]:
def generate_cat_columns(df_matches, n_bins):

    z_bins = np.linspace(-500, 500, n_bins)
    phot_bins = np.linspace(700,3000, n_bins)
    prob_bins = np.linspace(0.1,1.3, n_bins)


    df_matches['z_cat'] = None
    df_matches['phot_cat'] = None
    
    df_matches['GT_z_cat'] = None
    df_matches['GT_phot_cat'] = None


    df_matches['prob_cat'] = None


    for i in range(len(z_bins)-1):
        df_matches.loc[(df_matches['pred_z'] >= z_bins[i]) & (df_matches['pred_z'] <= z_bins[i+1]),'z_cat'] = z_bins[i]
        df_matches.loc[(df_matches['pred_phot'] >= phot_bins[i]) & (df_matches['pred_phot'] <= phot_bins[i+1]),'phot_cat'] = phot_bins[i]
        
        df_matches.loc[(df_matches['GT_z'] >= z_bins[i]) & (df_matches['GT_z'] <= z_bins[i+1]),'GT_z_cat'] = z_bins[i]
        df_matches.loc[(df_matches['GT_phot'] >= phot_bins[i]) & (df_matches['GT_phot'] <= phot_bins[i+1]),'GT_phot_cat'] = phot_bins[i]

        df_matches.loc[(df_matches['Prob_nms'] >= prob_bins[i]) & (df_matches['Prob_nms'] <= prob_bins[i+1]),'prob_cat'] = prob_bins[i]
    
    return df_matches


In [ ]:

sns.histplot(data=df_matches, x="pred_z", y='GT_z', bins=100)



In [216]:
sns.histplot(data=df_matches, x="Prob_nms", y='pred_z',hue='prob_cat', hue_order = prob_bins,bins=50)
#df_matches = df_matches[df_matches['Prob_nms'] < 0.5]


<Axes: xlabel='pred_z', ylabel='GT_z'>

In [37]:
labels_dict = {'z_cat':'Z (nm)', 'phot_cat': 'Photon count','GT_z_cat':'GT Z (nm)', 'GT_phot_cat': 'GT Photon count', 'z_error': 'z error (nm)', 'prob_cat':'Predicted probability', \
    'phot_error': 'photon count error', 'abs_z_error': 'absolute Z-error (nm)', 'abs_phot_error': 'absolute Photon count error', \
        'GT_z': 'ground truth Z (nm)', 'GT_phot': 'ground truth photon count', 'pred_z': 'predicted Z (nm)', 'pred_phot': 'predicted photon count'}

In [36]:
def plot_heatmap_mean(df, x_col, y_col, val_col, precision=0, extra_title='', count=False):

    if count:
        means = df.groupby([x_col, y_col]).count()[val_col].unstack(fill_value=0).stack()
    else:
        means = df.groupby([x_col, y_col]).mean()[val_col].unstack(fill_value=0).stack()
    
    means = means.sort_index()
    
    means_np = means.to_numpy()# Maybe add more bins and make sure axis are correct
    side_length = int(np.sqrt(means_np.shape[0]))
    
    fig, ax = plt.subplots()
    ar = np.array(means.index).reshape((side_length,side_length))
    x_axis_vals = np.array([val[0] for val in ar[:,0]])
    step = x_axis_vals[1] - x_axis_vals[0]
    x_axis_vals = (x_axis_vals + step//2).astype(int)


    y_axis_vals = np.array([val[1] for val in ar[0,::-1]])
    step = y_axis_vals[1] - y_axis_vals[0]
    y_axis_vals = (y_axis_vals - step//2).astype(int)

    data = np.flip(means_np.reshape((side_length,side_length)).T, axis=0)
    sns.heatmap(data, xticklabels=x_axis_vals, 
                yticklabels=y_axis_vals, annot=True, fmt=f".{precision}f", mask=(data==0))
    ax.set_xlabel(labels_dict[x_col])
    ax.set_ylabel(labels_dict[y_col])
    fig.suptitle(f"Mean {labels_dict[val_col]} for different photon counts and Z-positions \n {extra_title}")


In [354]:
df_matches = generate_cat_columns(df_matches, 15)

df_data = df_matches.copy()
#df_data = df_data[df_data['GT_phot']<1700]

plot_heatmap_mean(df_data, 'z_cat', 'phot_cat', 'GT_z')


/tmp/ipykernel_5067/3380770731.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  means = df.groupby([x_col, y_col]).mean()[val_col].unstack(fill_value=0).stack()


In [346]:
df_matches['prob_cat']

0        0.7
1       None
2       0.85
3       0.85
4       None
        ... 
9565     0.4
9566     0.4
9567     0.7
9568     0.4
9569     0.4
Name: prob_cat, Length: 9570, dtype: object

In [347]:
plt.close('all')

In [ ]:


phot_bins = list(range(700, 3100, 230))
phot_bins


In [209]:
plt.close('all')

In [275]:
fig = plt.figure()
sns.scatterplot(data=df_matches, x='GT_z', y='GT_phot', hue='z_error', palette='magma', hue_norm=(100, 300))

<Axes: xlabel='GT_z', ylabel='GT_phot'>

In [235]:
fig = plt.figure()
sns.scatterplot(data=df_matches, x="GT_z", y='GT_phot', hue='GT_z',size='GT_phot',  palette='coolwarm', alpha=0.5)


<Axes: xlabel='GT_z', ylabel='GT_phot'>

In [139]:
fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)
axs[0].scatter(pred_z, pred_ph, alpha=0.2)
axs[0].set_xlabel('Predicted z')
axs[0].set_ylabel('Predicted photon counts')

axs[1].scatter(gt_z, gt_ph, alpha=0.2)
axs[1].set_xlabel('Ground truth z')
axs[1].set_ylabel('Ground truth photon counts')


plt.show()

In [84]:
plt.figure()
plt.scatter(gt_z, gt_ph, alpha=0.2)
plt.xlabel('Ground truth z')
plt.ylabel('Ground truth photon counts')
plt.show()

In [73]:
plt.figure()
plt.scatter(gt_z, pred_z)
plt.xlabel('Ground truth z')
plt.ylabel('Prediction z')

Text(0, 0.5, 'Prediction z')

In [74]:
plt.figure()
plt.scatter(gt_ph, pred_ph)
plt.xlabel('Ground truth photon counts')
plt.ylabel('Prediction photon counts')

Text(0, 0.5, 'Prediction photon counts')

In [75]:
plt.figure()
plt.hist(gt_z - pred_z, bins=np.arange(-100, 100, 5))
plt.show()

In [76]:
plt.figure()
plt.hist(sampled_z, bins=np.arange(-500, 500, 10))
plt.show()

#### Predicted offsets

In [182]:
offsets_x = np.array([x[11] for x in preds_tmp])
offsets_y = np.array([x[12] for x in preds_tmp])
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].hist(offsets_x, bins=100,density=True)
ax[0].axvspan(offsets_x.min(), offsets_x.max(), color='green', alpha=0.1)
ax[0].set_xlabel('offsets_x')
ax[0].set_ylabel('counts')
ax[0].set_title(f"Mean offset_x is {np.mean(offsets_x):0.3f}, min:{np.min(offsets_x):0.3f}, max:{np.max(offsets_x):0.3f}")

ax[1].hist(offsets_y, bins=100, density=True)
ax[1].axvspan(offsets_y.min(), offsets_y.max(), color='green', alpha=0.1)
ax[1].set_xlabel('offsets_y')
ax[1].set_ylabel('counts')
ax[1].set_title(f"Mean offset_y is {np.mean(offsets_y):0.3f}, min:{np.min(offsets_y):0.3f}, max:{np.max(offsets_y):0.3f}")


fig.suptitle('Offsets predicted by the network')
plt.tight_layout()
plt.show()

In [183]:
gt_z.min(), gt_z.max()

type: name 'gt_z' is not defined

In [33]:
plt.figure()
plt.hist(gt_z, bins=np.arange(-500, 500, 20), label='ground truth', alpha=0.5)
plt.hist(pred_z, bins=np.arange(-500, 500, 20), label='predicted', alpha=0.5)

plt.legend(loc='upper right')
plt.figure('')
plt.show()

In [34]:
plt.figure()
plt.hist(pred_z, bins=np.arange(-500, 500, 20), label='predicted')
plt.legend(loc='upper right')
plt.show()

In [36]:
plt.figure()
plt.hist(gt_ph, bins=np.arange(700, 3000, 20), label='ground truth', alpha=0.4)
plt.hist(pred_ph, bins=np.arange(700, 3000, 20), label='predicted', alpha=0.4)

plt.legend(loc='upper right')
plt.show()

In [37]:
plt.figure()
plt.hist(sampled_ph, bins=np.arange(700, 3000, 10), label='sampled during training')
plt.show()

```
List of all matches localizations for further evaluation in format: 
    1. x_gt
    2. y_gt
    3. z_gt
    4. intensity_gt,
    5. x_pred,
    6. y_pred
    7. z_pred
    8. intensity_pred
    9. nms_p
    10.x_sig
    11.y_sig
    12.z_sig
```

In [38]:
sigma_x = np.array([x[7] for x in preds_tmp])
sigma_y = np.array([x[8] for x in preds_tmp])
sigma_z = np.array([x[9] for x in preds_tmp])
sigma_ph = np.array([x[10] for x in preds_tmp])

In [39]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(sigma_x, bins=25)
ax[0, 0].axvspan(sigma_x.min(), sigma_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$\sigma_x$ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean sigma_x is {np.mean(sigma_x):0.3f}, min:{np.min(sigma_x):0.3f}, max:{np.max(sigma_x):0.3f}")

ax[0, 1].hist(sigma_y, bins=25)
ax[0, 1].axvspan(sigma_y.min(), sigma_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$\sigma_y$ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean sigma_y is {np.mean(sigma_y):0.3f}, min:{np.min(sigma_y):0.3f}, max:{np.max(sigma_y):0.3f}")


ax[1, 0].hist(sigma_z, bins=25)
ax[1, 0].axvspan(sigma_z.min(), sigma_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$\sigma_z$ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean sigma_z is {np.mean(sigma_z):0.3f}, min:{np.min(sigma_z):0.3f}, max:{np.max(sigma_z):0.3f}")

ax[1, 1].hist(sigma_ph, bins=25)
ax[1, 1].axvspan(sigma_ph.min(), sigma_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$\sigma_{ph}$ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean sigma_ph is {np.mean(sigma_ph):0.3f}, min:{np.min(sigma_ph):0.3f}, max:{np.max(sigma_ph):0.3f}")

fig.suptitle('Sigmas predicted by the network')
plt.tight_layout()
plt.show()

#### RMSE values distributions

In [40]:
residual_x = np.abs(gt_x - pred_x)
residual_y = np.abs(gt_y - pred_y)
residual_z = np.abs(gt_z - pred_z)
residual_ph = np.abs(gt_ph - pred_ph)

In [41]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(residual_x, bins=50)
ax[0, 0].axvspan(residual_x.min(), residual_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$Absoulte distance x $ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean {np.mean(residual_x):0.3f}, median: {np.median(residual_x):0.3f}, min:{np.min(residual_x):0.3f}, max:{np.max(residual_x):0.3f}")

ax[0, 1].hist(residual_y, bins=50)
ax[0, 1].axvspan(residual_y.min(), residual_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$Absoulte distance y $ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean dist y {np.mean(residual_y):0.3f}, median: {np.median(residual_y):0.3f}, min:{np.min(residual_y):0.3f}, max:{np.max(residual_y):0.3f}")

ax[1, 0].hist(residual_z, bins=50)
ax[1, 0].axvspan(residual_z.min(), residual_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$Absoulte distance z $ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean dist z {np.mean(residual_z):0.3f}, median: {np.median(residual_z):0.3f}, min:{np.min(residual_z):0.3f}, max:{np.max(residual_z):0.3f}")

ax[1, 1].hist(residual_ph, bins=50)
ax[1, 1].axvspan(residual_ph.min(), residual_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$Absoulte difference Photon counts $ ADU')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean diff photon counts {np.mean(residual_ph):0.3f}, median: {np.median(residual_ph):0.3f}, min:{np.min(residual_ph):0.3f}, max:{np.max(residual_ph):0.3f}")

fig.suptitle('Absolute distance btn prediction and ground truth')

plt.tight_layout()
plt.show()

### Real images

In [14]:
import skimage.io as sio

In [120]:


venus_images = Path('/home/axtr7550/Chromosome_project/real_images/EXP-23-CA3051/Run/Pos01/fluor')
venus_filenames = sorted(list(venus_images.glob('*.tiff')))

In [121]:
eval_imgs_all = []
for filename in venus_filenames:
    eval_imgs_all.append(sio.imread(filename))

In [122]:
eval_imgs_all = np.stack(eval_imgs_all)
eval_imgs_all = eval_imgs_all[:,eval_imgs_all.shape[1]//2:,:]

In [123]:
eval2 = eval_imgs_all
fov_size = [eval_imgs_all.shape[2] * 65, eval_imgs_all.shape[1] * 65]
plot_num = None
preds_raw, n_per_img, plot_area = recognition(model=chromo_model, eval_imgs_all=eval2, batch_size=32, use_tqdm=False,
                                     nms=True, candidate_threshold=0.4,
                                     nms_threshold=0.3,
                                     pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                     plot_num=1,
                                     win_size=chromo_model.data_generator.simulation_params['train_size'],
                                     padding=True,
                                     padded_background=chromo_model.simulation_params['bg_values'])

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]


In [125]:
df_real_raw_predictions = pd.DataFrame(preds_raw, columns=['id', 'image_id', 'x_pos', 'y_pos', 'z_pos', 'phot', \
    'prob', 'x_sigma', 'y_sigma', 'z_sigma', 'phot_sigma', 'x_offset', 'y_offset'])

df_real_raw_predictions['Shifted'] = False

In [126]:
df_all_predictions = pd.concat([df_all_raw_predictions, df_real_raw_predictions], axis=0)


,id,image_id,x_pos,y_pos,z_pos,phot,prob,x_sigma,y_sigma,z_sigma,phot_sigma,x_offset,y_offset,Shifted
0,1.0,1.0,23334.234375,5533.872559,31.059120,2860.272217,1.119140,17.884209,16.195568,50.963375,114.011665,-0.011784,0.136504,True
1,2.0,1.0,24488.792969,5530.675293,-150.632751,2904.697021,1.072772,11.055288,10.467881,29.955072,65.241516,-0.249340,0.087315,True
2,3.0,1.0,22645.369141,5993.561523,-96.295471,2346.665039,1.054051,10.639866,10.628008,34.833584,151.078766,0.390285,0.208639,True
3,3.0,1.0,26177.916016,5595.559570,-213.445328,2382.603760,0.815142,14.678159,14.735226,39.936417,233.631943,-0.262835,0.085532,True
4,5.0,1.0,27396.222656,5855.624512,-190.002457,2606.810303,0.911270,16.129734,15.610109,43.968723,237.849197,0.480362,0.086528,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137520,3255.0,301.0,70016.773438,61605.453125,93.185905,2841.809326,1.074650,9.984779,9.549804,35.858582,79.428749,0.181184,-0.223817,False
137521,3256.0,301.0,73176.492188,61750.589844,-201.884171,1209.964844,0.888641,19.357586,19.213541,54.140194,109.232674,-0.207807,0.009056,False
137522,3257.0,301.0,73561.492188,61832.851562,-22.121204,1256.196411,0.995493,17.842817,17.946295,56.964394,103.063560,-0.284735,0.274615,False
137523,3258.0,301.0,71365.710938,61872.406250,-22.840311,1202.854370,0.749299,20.008411,20.700874,85.047882,115.764786,-0.066046,-0.116810,False


In [129]:
df_all_predictions.to_csv('/home/axtr7550/Chromosome_project/real_images/EXP-23-CA3051/Run/all_predictions_700_1+9.csv')


In [34]:
fig, ax = plt.subplots()
data_df =df_all_predictions[df_all_predictions['Shifted'] == False]
sns.scatterplot(data=data_df, x="z_pos", y="phot", hue='prob', alpha=0.5)
ax.set_title('Not Shifted')


Text(0.5, 1.0, 'Not Shifted')

In [47]:
df_real_filtered_preds = df_real_raw_predictions[df_real_raw_predictions['prob'] > 0.5]


In [108]:
path = 'C:/Users/Raket/OneDrive - Uppsala universitet/Desktop/Sommarjobb/Chromosome_project/data/'
df_matches = pd.read_csv(path+'rotated_matches.csv')
df_predictions = pd.read_csv(path+'rotated_predictions.csv')


In [28]:
n_bins = 20
df_matches = generate_cat_columns(df_matches, n_bins)
df_matches


,GT_x,GT_y,GT_z,GT_phot,pred_x,pred_y,pred_z,pred_phot,Prob_nms,z_sigma,Unnamed: 10,Unnamed: 11,z_cat,phot_cat,GT_z_cat,GT_phot_cat,prob_cat
0,68060.220947,29715.564880,369.151652,2674.303293,68060.914062,29714.609375,368.453857,2380.373779,0.844238,19.502956,203.809799,0.090998,342.105263,2273.684211,342.105263,2636.842105,0.794737
1,44706.249390,53046.256409,55.104613,1336.189091,44705.843750,53047.464844,70.424889,1752.637817,1.033803,26.161909,153.927399,-0.217815,26.315789,1668.421053,26.315789,1305.263158,0.984211
2,68936.927490,53305.169373,99.094450,1582.688928,68937.781250,53304.140625,132.131653,1755.289673,0.956463,24.977852,152.128922,-0.418763,131.578947,1668.421053,78.947368,1547.368421,0.921053
3,35109.136658,30137.229767,171.535552,2567.669749,35107.785156,30136.357422,130.372009,2368.313232,0.951708,17.813414,135.111771,0.119775,78.947368,2273.684211,131.578947,2515.789474,0.921053
4,68135.845337,23987.737427,84.488273,2717.178404,68135.085938,23989.726562,80.716721,2774.395264,1.090230,20.246861,116.764404,0.232144,78.947368,2757.894737,78.947368,2636.842105,1.047368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9565,40973.303833,29837.763214,-363.933325,831.501782,41026.429688,29950.322266,-416.742615,885.396729,0.485788,31.305876,83.838692,0.175822,-447.368421,821.052632,-394.736842,821.052632,0.478947
9566,56763.804932,35625.526428,-493.031621,1003.791958,56788.562500,35749.699219,-424.411011,901.603027,0.482002,36.610123,98.300919,-0.329804,-447.368421,821.052632,-500.0,942.105263,0.478947
9567,40072.825317,15585.887375,-491.915911,1497.940779,40100.089844,15718.686523,-445.896057,1108.504028,0.735024,23.420807,125.818848,-0.075551,-447.368421,1063.157895,-500.0,1426.315789,0.731579
9568,31090.030823,35736.134338,409.530401,2684.216917,31219.544922,35817.281250,277.526154,2709.803223,0.511225,28.247869,190.125595,0.300697,236.842105,2636.842105,394.736842,2636.842105,0.478947


In [ ]:
z_bins = np.linspace(-500, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)
prob_bins = np.linspace(0.1,1.3, n_bins)


df_matches['z_cat'] = None
df_matches['phot_cat'] = None

df_matches['GT_z_cat'] = None
df_matches['GT_phot_cat'] = None


df_matches['prob_cat'] = None


for i in range(len(z_bins)-1):    
    df_matches.loc[(df_matches['GT_z'] >= z_bins[i]) & (df_matches['GT_z'] <= z_bins[i+1]),'GT_z_cat'] = z_bins[i]
    df_matches.loc[(df_matches['GT_phot'] >= phot_bins[i]) & (df_matches['GT_phot'] <= phot_bins[i+1]),'GT_phot_cat'] = phot_bins[i]



In [109]:
from itertools import product
n_bins = 21
z_bins = np.linspace(-500, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)

all_pdfs = np.zeros((len(z_bins)-1, len(phot_bins)-1, len(z_bins)-1, len(phot_bins)-1))

squares = list(product(range(n_bins-1), range(n_bins-1)))

for i, j in squares:
    
    df_filtered = df_matches.loc[(df_matches['GT_z'] >= z_bins[i]) & (df_matches['GT_z'] <= z_bins[i+1]),:]
    df_filtered = df_filtered.loc[(df_filtered['GT_phot'] >= phot_bins[j]) & (df_filtered['GT_phot'] <= phot_bins[j+1]),:]


    hist, _, _ = np.histogram2d(df_filtered['pred_z'], df_filtered['pred_phot'], bins=(z_bins,phot_bins), density=True)

    all_pdfs[i,j,:,:] = hist


In [180]:
## Change GT distribution 
z_mean = -200
z_std = 100
phot_mean = 1000
phot_std = 200
z_values = np.random.normal(z_mean,  z_std, 1000)
phot_values = np.random.normal(phot_mean, phot_std, 1000)

phot_values = np.clip(phot_values, 701, 2999)
z_values = np.clip(z_values, -499, 499)


In [181]:
z_indeces = (((z_values+500)*n_bins)/1000).astype(int)
phot_indeces = (((phot_values-700)*n_bins)/2300).astype(int)
indeces = np.stack((z_indeces, phot_indeces), axis=1)
indeces


array([[7, 1],
       [4, 3],
       [7, 1],
       ...,
       [3, 0],
       [4, 3],
       [7, 6]])

In [182]:
votes,_,_ = np.histogram2d(z_indeces, phot_indeces, bins =(np.arange(n_bins), np.arange(n_bins)))
fig = plt.figure()

sns.heatmap(np.flip(votes.T, axis=0), annot=True) 



<Axes: >

In [43]:
dummy = np.zeros((20,20))
plt.figure()
sns.heatmap(votes)

dummy[10, 2] = 1
plt.figure()
sns.heatmap(dummy, annot=True, cmap="Blues")
plt.figure()
sns.heatmap(all_pdfs[10,2,...])

<Axes: >

In [183]:
all_predictions = np.zeros((n_bins-1,n_bins-1))

for i, j in squares:
    n = int(votes[i,j])
    vals = np.array(random.choices(np.arange((n_bins-1)**2), weights=all_pdfs[i,j,:,:].reshape((-1)), k=n))
    pred_votes, _, _ = np.histogram2d(vals//(n_bins-1), vals%(n_bins-1), bins = (np.arange(n_bins), np.arange(n_bins)))
    all_predictions += pred_votes



In [184]:
fig, ax = plt.subplots()

sns.heatmap(np.flip(all_predictions.T, axis=0), annot=True)
ax.set_title(f"Predictions, z mean={z_mean}, z std={z_std} \n  phot mean={phot_mean}, phot std={phot_std}")

Text(0.5, 1.0, 'Predictions, z mean=-200, z std=100 \n  phot mean=1000, phot std=200')

<Axes: >

In [32]:
labels_dict['z_sigma'] = 'z sigma'

plot_heatmap_mean(df_matches, 'z_cat', 'phot_cat', 'GT_z', precision=1)

In [ ]:


fig, ax = plt.subplots()
sns.scatterplot(data=df_real_filtered_preds, x="z_pos", y="phot", hue='prob', alpha=0.5)




In [43]:
n_bins = 20
z_bins = np.linspace(-500, 500, n_bins)
phot_bins = np.linspace(700,3000, n_bins)
prob_bins = np.linspace(0.5,1, n_bins)


df_all_predictions['z_cat'] = None
df_all_predictions['phot_cat'] = None


df_all_predictions['prob_cat'] = None


fig, ax = plt.subplots()
for i in range(len(z_bins)-1):
    df_all_predictions.loc[(df_all_predictions['z_pos'] >= z_bins[i]) & (df_all_predictions['z_pos'] <= z_bins[i+1]),'z_cat'] = z_bins[i]
    df_all_predictions.loc[(df_all_predictions['phot'] >= phot_bins[i]) & (df_all_predictions['phot'] <= phot_bins[i+1]),'phot_cat'] = phot_bins[i]
    

    df_all_predictions.loc[(df_all_predictions['prob'] >= prob_bins[i]) & (df_all_predictions['prob'] <= prob_bins[i+1]),'prob_cat'] = prob_bins[i]


In [113]:
labels_dict['prob'] = 'Predicted probability'
labels_dict['z_sigma'] = 'z sigma'
labels_dict['phot_sigma'] = 'z sigma'


data_df = df_all_predictions[df_all_predictions['Shifted']==False]
plot_heatmap_mean(data_df, 'z_cat', 'phot_cat', 'phot_sigma', precision=3, extra_title='Not Shifted')


/tmp/ipykernel_50486/1328008263.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  means = df.groupby([x_col, y_col]).mean()[val_col].unstack(fill_value=0).stack()


In [111]:
labels_dict['count'] = 'count'
data_df = df_all_predictions[df_all_predictions['Shifted']]
plot_heatmap_mean(data_df, 'z_cat', 'phot_cat', 'z_sigma', count=True, extra_title='Shifted')

data_df = df_all_predictions[df_all_predictions['Shifted']==False]
plot_heatmap_mean(data_df, 'z_cat', 'phot_cat', 'z_sigma', count=True, extra_title='Not shifted')



In [52]:
data_df = df_real_raw_predictions[(df_real_raw_predictions['phot'] >2400) & (df_real_raw_predictions['z_pos'] < 150) \
    &(df_real_raw_predictions['z_pos'] > -100)] 

In [53]:
fig, ax = plt.subplots()
sns.scatterplot(data=data_df, x="z_pos", y="phot", hue='z_sigma')

<Axes: xlabel='z_pos', ylabel='phot'>